## 데이터 준비

In [1]:
import pandas as pd
df = pd.read_csv('imdb.zip')

In [2]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [3]:
import joblib
tk = joblib.load('tokenizer.pkl')

In [4]:
from sklearn.model_selection import train_test_split
review_train, review_test, y_train, y_test = train_test_split(df['review'],\
                            df['sentiment'], test_size=0.2, random_state=42)

In [5]:
# 토큰화한다
seqs = tk.texts_to_sequences(review_train)

In [6]:
review_train.iloc[0]

'It is an insane game.'

In [7]:
seqs[0]

[9, 6, 33, 1258, 214]

In [8]:
seqs[:3]

[[9, 6, 33, 1258, 214],
 [178, 5, 28, 35, 23, 168, 713, 591, 3, 713, 1, 10, 1, 280],
 [206, 336, 4]]

## 순방향 순환신경망

In [9]:
import tensorflow as tf

In [10]:
# maxlen : None(기본값), 10 : 10으로 문장의 길이를 잘라서 패딩처리
# padding : pre(기본값), post(뒤쪽에 0 패딩처리), pre 권장
# truncating : pre(기본값), maxlen 값이 설정이 되야지 동작,성능 테스트 후 적당한걸 선택 
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, maxlen=None,\
                            padding='pre', truncating='pre')

In [11]:
len(pads), pads.shape

(800, (800, 73))

In [12]:
NUM_WORDS= tk.num_words + 1

## 순방향 순환신경망

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 8)           16008     
                                                                 
 lstm (LSTM)                 (None, 8)                 544       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 10ms/step - loss: 0.6920 - accuracy: 0.5575
Epoch 2/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6864 - accuracy: 0.7312
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6713 - accuracy: 0.7912
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6222 - accuracy: 0.8125
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.4878 - accuracy: 0.8600
Epoch 6/10
25/25 [==============================] - 0s 11ms/step - loss: 0.3819 - accuracy: 0.8838
Epoch 7/10
25/25 [==============================] - 0s 11ms/step - loss: 0.3012 - accuracy: 0.9300
Epoch 8/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2438 - accuracy: 0.9500
Epoch 9/10
25/25 [==============================] - 0s 10ms/step - loss: 0.1981 - accuracy: 0.9688
Epoch 10/10
25/25 [==============================] - 0s 10ms/step - loss: 0.1630 - accuracy: 0.9737


## 역방향 순환신경망

### pre padding 적용

In [17]:
# 패딩을 post로 처리하는 것이 좋음, 순방향일 경우는 pre로 처리하는 것을 권장
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [18]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [19]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 11ms/step - loss: 0.6920 - accuracy: 0.5200
Epoch 2/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6875 - accuracy: 0.6137
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6777 - accuracy: 0.7275
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6470 - accuracy: 0.7925
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.5432 - accuracy: 0.8537
Epoch 6/10
25/25 [==============================] - 0s 10ms/step - loss: 0.4188 - accuracy: 0.8888
Epoch 7/10
25/25 [==============================] - 0s 10ms/step - loss: 0.3401 - accuracy: 0.9112
Epoch 8/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2713 - accuracy: 0.9388
Epoch 9/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2581 - accuracy: 0.9362
Epoch 10/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2289 - accuracy: 0.9550


### post padding 적용

In [20]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs, padding='post')

In [21]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [22]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 10ms/step - loss: 0.6929 - accuracy: 0.5038
Epoch 2/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6883 - accuracy: 0.6687
Epoch 3/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6756 - accuracy: 0.7525
Epoch 4/10
25/25 [==============================] - 0s 10ms/step - loss: 0.6261 - accuracy: 0.7925
Epoch 5/10
25/25 [==============================] - 0s 10ms/step - loss: 0.5299 - accuracy: 0.8512
Epoch 6/10
25/25 [==============================] - 0s 10ms/step - loss: 0.4262 - accuracy: 0.8888
Epoch 7/10
25/25 [==============================] - 0s 10ms/step - loss: 0.3443 - accuracy: 0.9262
Epoch 8/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2754 - accuracy: 0.9425
Epoch 9/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2671 - accuracy: 0.9350
Epoch 10/10
25/25 [==============================] - 0s 10ms/step - loss: 0.2388 - accuracy: 0.9463


## 양방향 순환신경망

In [24]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [25]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 8)           16008     
                                                                 
 bidirectional (Bidirectiona  (None, 16)               1088      
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 17,113
Trainable params: 17,113
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [30]:
model.fit(pads, y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 4s 12ms/step - loss: 0.6930 - accuracy: 0.4925
Epoch 2/10
25/25 [==============================] - 0s 11ms/step - loss: 0.6869 - accuracy: 0.7300
Epoch 3/10
25/25 [==============================] - 0s 11ms/step - loss: 0.6733 - accuracy: 0.7850
Epoch 4/10
25/25 [==============================] - 0s 11ms/step - loss: 0.6219 - accuracy: 0.8087
Epoch 5/10
25/25 [==============================] - 0s 11ms/step - loss: 0.4981 - accuracy: 0.8637
Epoch 6/10
25/25 [==============================] - 0s 12ms/step - loss: 0.3422 - accuracy: 0.9112
Epoch 7/10
25/25 [==============================] - 0s 11ms/step - loss: 0.2432 - accuracy: 0.9400
Epoch 8/10
25/25 [==============================] - 0s 11ms/step - loss: 0.1920 - accuracy: 0.9600
Epoch 9/10
25/25 [==============================] - 0s 11ms/step - loss: 0.1688 - accuracy: 0.9613
Epoch 10/10
25/25 [==============================] - 0s 11ms/step - loss: 0.1319 - accuracy: 0.9675
